# ***This code aims to solve the promblem 10.2 as well as get familiar with forward-backward algorithm in HMM***

In [89]:
import numpy

In [90]:
def forward(O, A, B, pi):
    """ back_alg iterator

    Args:
        O ([list]): contains observed sequence, of shape (T)
        A ([matrix]): aij for prob that convert from status i at time-step t to status j at time-step t+1, of shape (N, N)
        B ([matrix]): bij for at time-step t, status i yields observe output j, of shape (N, n)
        pi ([ndarray]): PIi for prob of status i serves as initial status, of shape (1, N)

    Returns:
        [matrix]: all-time alpha-matrix
        [float]: condition probability of O given A, B, PI
    """
    T = len(O)
    N = A.shape[0]
    alpha = np.zeros((N, T))
    alpha[:, 0] = (pi.T * B[:, O[0]]).diagonal()        # (3, 1) * (3, 1) = (3, 3), elementwise products lay in diagonal
    for t in range(T - 1):
        o = O[t + 1]
        alpha[:, t + 1] = forward_step(o, A, B, alpha[:, t])
    prob = sum(alpha[:, -1])
    return alpha, prob

In [91]:
def forward_step(o, A, B, prev_alpha):
    """compute alpha[t + 1]

    Args:
        o ([int]): observe at time-step t+1
        A ([matrix]): aij for prob that convert from status i at time-step t to status j at time-step t+1
        B ([matrix]): bj for prob that status j yields obseve at time-step t+1
        prev_alpha ([matrix]): alpha sequence at time-step t+1

    Returns:
        [matrix]: alpha at time-step t+1
    """
    b = B[:, o]
    alpha = (A.T).dot(prev_alpha) * b
    return alpha

In [92]:
A = np.array([[0.5, 0.1, 0.4],
              [0.3, 0.5, 0.2],
              [0.2, 0.2, 0.6]])

B = np.array([[0.5, 0.5],
              [0.4, 0.6],
              [0.7, 0.3]])

pi = np.array([[0.2, 0.3, 0.5]])

O = [0, 1, 0, 0, 1, 0, 1, 1]

In [93]:
alpha, prob = forward(O, A, B, pi)
print(prob)
print(alpha)

0.0034767094492823996
[[0.1        0.078      0.04032    0.0208668  0.0114321  0.00549669
  0.00281056 0.00132502]
 [0.12       0.084      0.026496   0.01236192 0.01019392 0.00338373
  0.00245952 0.00121063]
 [0.35       0.0822     0.068124   0.04361112 0.01109573 0.00928834
  0.00253453 0.00094105]]


In [94]:
from HMM_utils import *

In [95]:
beta, prob_b = backward(O, A, B, pi)
print(prob_b)
print(beta)

[0.00347671]
[[0.00632569 0.01482964 0.02556442 0.04586531 0.105521   0.1861
  0.43       1.        ]
 [0.00684706 0.01227214 0.02343448 0.05280909 0.100883   0.2415
  0.51       1.        ]
 [0.00577855 0.01568294 0.02678985 0.04280618 0.111934   0.1762
  0.4        1.        ]]


In [101]:
def status_prob(alpha, beta, *Args):
    """compute probability of status i at time-step t

    Args:
        alpha ([matrix]): alpha of all status of all time-steps
        beta ([matrix]): beta of all status of all time-steps
        i ([int]): index of status 
        t ([int]): index of time-step

    Returns:
        [float]: probability of status i at time-step t
    """
    if (Args):
        i, t = Args
        ## return prob at certain i and t
        return alpha[i, t] * beta[i, t] / np.expand_dims(alpha[:, t], axis = 0).dot(np.expand_dims(beta[:, t], axis = 1))

    else:
        N = alpha.shape[0]
        stat_prob = np.zeros((N, N))
        stat_prob = np.multiply(alpha, beta)
        total_prob = np.sum(stat_prob, axis = 0)
        stat_prob /= total_prob
        ## return the whole
        return stat_prob

In [102]:
prob_4_3 = status_prob(alpha, beta, 2, 3)
print(prob_4_3)
stat_prob = status_prob(alpha, beta)
print(stat_prob[2, 3])

[[0.53695182]]
0.5369518160647323
